In [143]:
%load_ext autoreload
%autoreload 2
from app.config_loader import load_config_once
conf = load_config_once(output_folder="SIDKIM")
import pdfplumber, camelot, pprint, json5,json, os, re
import fitz #type: ignore
import pandas as pd
import numpy as np

from app.fund_sid_data import *

with open("config\\sid_params.json5","r+") as file:
    params = json5.load(file)

# with open("config\\sid_params.json5","r+") as file:
#     params = json5.dump(params,file)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Config already loaded. Skipping re-initialization.


In [ ]:
path = os.path.join(os.getcwd(),"pdfs","96_17078_Apr-2025_1746183006_SID.pdf")
object = AngelOne(amc_id="96",path=path)
temp_dict = {}
temp_dict.update(object._parse_page_zero([0]))
temp_dict.update(object._parse_scheme_table_data("5,6,7,8,9,10"))
temp_dict.update(object._parse_fund_manager_info("16,17"))

In [150]:
object = AngelOne(amc_id="96",path=path)
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)
with open("tempor.json","w+") as file:
    json.dump(dfs,file)

Function Running: refine_extracted_data
Function Running: merge_and_select_data


In [130]:
with open("tempor.json","w+") as file:
    json.dump(temp_dict,file)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","unifi1747199156275.pdf")
object = Unifi(amc_id=97, path=path)


In [ ]:
def _extract_managerial_data(self, main_key: str, data, pattern: str):
        # print("running manager data")
        final_list = []
        manager_regex = self.REGEX[pattern]
        if not isinstance(data,list):
            print("_extract_manager_data-> data not list")
            return list(data)
        
        for manager in data:
            temp = {}
            for key, val in manager.items():
                clean_val = re.findall(manager_regex[key],val,re.IGNORECASE)
                update_val = " ".join(clean_val)
                temp[key] = update_val
            if temp["name"]:
                final_list.append(temp)
        return {"fund_manager": final_list}

In [121]:
path = os.path.join(os.getcwd(),"pdfs","unifi1747199772321.pdf")
path2 = os.path.join(os.getcwd(),"pdfs","SID-Angel-One-Nifty-50-Index-Fund-2.pdf")

In [ ]:
import pdfplumber
import pandas as pd


pdf_path = "your_file.pdf"
table_bbox = (0, 190, 600, 500)  # (x0, top, x1, bottom)

table_df = None
text_lines = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        cropped = page.within_bbox(table_bbox)
        table = cropped.extract_table()
        if table:
            table_df = pd.DataFrame(table[1:], columns=table[0])

        full_text = page.extract_text()
        if full_text:
            text_lines.extend(full_text.splitlines())

# Now you have:
# table_df -> the DataFrame containing the table
# text_lines -> list of all text lines on the page(s)


In [122]:
scheme_tables = camelot.read_pdf(path, pages="1", flavor='lattice',table_areas = ["20, 190, 580, 600"])
df = pd.concat([table.df for table in scheme_tables], ignore_index=True)

In [128]:
df.to_excel("first_page.xlsx")

In [ ]:
def _parse_page_zero(path:str,amc:str,pages:list):
    final_dict = {}
    bbox_text = []
    other_text = []
    risk_bbox = params[amc]["PARAMS"]["page_zero"]["bbox"]
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            if pgn in pages:
                bbox_text.append(page.get_text("text", clip=risk_bbox).replace("\n", ""))
                other_text.append(page.get_text("text").replace("\n", ""))
        final_dict["risk_bbox"] = bbox_text
        final_dict["other_text"] = other_text
    return final_dict

def _detect_column_start_by_keywords(df,threshold,keywords): 
    for i in range(df.shape[1]):
        col = df.iloc[:, i]
        col_cleaned = col.dropna().astype(str)
        col_cleaned = col_cleaned.apply(lambda x: re.sub(r"\s+", " ", x).strip())
        col_cleaned = col_cleaned[col_cleaned.str.strip() != ""]

        count = sum(any(kw in cell for kw in keywords) for cell in col_cleaned)

        if count >= threshold:
            return i
    return 1

def _parse_scheme_table_data(path:str,amc:str,pages:str):
    final_dict = {}
    
    threshold = params[amc]["PARAMS"]["table_data"]["threshold"]
    keywords = params[amc]["PARAMS"]["table_data"]["keywords"]
    
    scheme_tables = camelot.read_pdf(path, pages=pages, flavor='lattice')
    
    dfs = pd.concat([table.df for table in scheme_tables], ignore_index=True)
    col_start = _detect_column_start_by_keywords(dfs,threshold=threshold,keywords=keywords)
    dfs = dfs.iloc[:,col_start:]
    # print(dfs.head(20))
    dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x) #applies to dataframe not series
    dfs.iloc[:, 0] = dfs.iloc[:, 0].replace(r'^\s*$', np.nan, regex=True)
    dfs.iloc[:, 0] = dfs.iloc[:, 0].ffill() #fill corresponding structure
    dfs.columns = ['Title'] + [f'Data{i}' for i in range(1, dfs.shape[1])] #new col structure
    
    data_cols = [col for col in dfs.columns if col != 'Title']
    for title, group_df in dfs.groupby('Title', sort=False):
        title = re.sub(r"[^A-Za-z0-9\s]+","",title).strip()
        title = re.sub(r"\s+"," ",title).strip()
        title = "_".join(title.lower().split(" "))
        
        current_title_values = []
        for index, row in group_df.iterrows():
            for col_name in data_cols:
                cell_value = row[col_name]
                if isinstance(cell_value, str) and cell_value.strip() != '':
                    current_title_values.append(cell_value)
        final_dict[title] = current_title_values
        
    return final_dict

def _parse_fund_manager_info(path:str,amc:str,pages:str):
    
    row_count = params[amc]["PARAMS"]["manager_data"]["row_count"]
    match_order = params[amc]["PARAMS"]["manager_data"]["order"]
    manager_tables = camelot.read_pdf(path, pages=pages, flavor='lattice')
    dfs = pd.concat([table.df for table in manager_tables], ignore_index=True)
    dfs = dfs.applymap(lambda x: x.replace('\n', ' ') if isinstance(x, str) else x)
    
    data_rows = [list(row) for _, row in dfs.iterrows() if row.count()==row_count]
    manager_list = []
    for data_r in data_rows:
        manager_list.append({
            "name":data_r[match_order["name"]],
            "experience":data_r[match_order["experience"]],
            "qualification":data_r[match_order["qualification"]],
        })
    return  {"fund_manager": manager_list}

In [ ]:
temp_dict = {}
temp_dict.update(_parse_page_zero(path,"59",[0]))
temp_dict.update(_parse_scheme_table_data(path,"59","5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21"))
temp_dict.update(_parse_fund_manager_info(path,"59","29,30"))

In [ ]:
temp_dict = {}
temp_dict.update(_parse_page_zero(path2,"96",[0]))
temp_dict.update(_parse_scheme_table_data(path2,"96","5,6,7,8,9,10"))
temp_dict.update(_parse_fund_manager_info(path2,"96","16,17"))

In [162]:
with open("tempor.json","w+") as file:
    json.dump(temp_dict,file)